In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings as wn
import sklearn.preprocessing as skpre
import category_encoders as ce

wn.simplefilter( "ignore" )

# Preparamos los Sets de Training

In [2]:
loc_ts = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning\Training Sets"

In [3]:
ins_18_20 = pd.read_csv( loc_ts + "\\ins_18_20.csv" ); 
ins_21_23 = pd.read_csv( loc_ts + "\\ins_21_23.csv" ); 
ins_24_26 = pd.read_csv( loc_ts + "\\ins_24_26.csv" ); 

In [5]:
auc_18_20 = pd.read_csv( loc_ts + "\\auc_18_20.csv" ); 
auc_21_23 = pd.read_csv( loc_ts + "\\auc_21_23.csv" );
auc_24_26 = pd.read_csv( loc_ts + "\\auc_24_26.csv" );

# Genero los Sets con el primer install

In [7]:
ins_18_20["date"] =  pd.to_datetime( ins_18_20["date"] ); ins_18_20["_sc"] = ins_18_20["date"] - dt.datetime( year = 2019, month = 4, day = 18 ); ins_18_20["_sc"] = ins_18_20["_sc"].dt.total_seconds();
ins_21_23["date"] =  pd.to_datetime( ins_21_23["date"] ); ins_21_23["_sc"] = ins_21_23["date"] - dt.datetime( year = 2019, month = 4, day = 21 ); ins_21_23["_sc"] = ins_21_23["_sc"].dt.total_seconds(); 
ins_24_26["date"] =  pd.to_datetime( ins_24_26["date"] ); ins_24_26["_sc"] = ins_24_26["date"] - dt.datetime( year = 2019, month = 4, day = 24 ); ins_24_26["_sc"] = ins_24_26["_sc"].dt.total_seconds(); 

In [8]:
ins_18_20_first_ins = ins_18_20.sort_values( by = ["ref_hash","_sc"], ascending = True ).drop_duplicates( subset = ["ref_hash"], keep = "first" )
ins_21_23_first_ins = ins_21_23.sort_values( by = ["ref_hash","_sc"], ascending = True ).drop_duplicates( subset = ["ref_hash"], keep = "first" )
ins_24_26_first_ins = ins_24_26.sort_values( by = ["ref_hash","_sc"], ascending = True ).drop_duplicates( subset = ["ref_hash"], keep = "first" )

# Sacamos los Features

In [9]:
loc_ftr = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning\Features\ftr_ins"

In [10]:
rh_18_20 = ins_18_20_first_ins[ ["ref_hash"] ] #RH unicos que coinciden con los datos que usamos para entrenar
rh_21_23 = ins_21_23_first_ins[ ["ref_hash"] ] #RH unicos que coinciden con los datos que queremos predecir

tg_18_20 = ins_18_20_first_ins[ ["_sc"] ]
tg_21_23 = ins_21_23_first_ins[ ["_sc"] ]

## <span style="color:red"> **Hora mas popular del usuario en las subastas** </span> 

Le aplicamos la transformacion sin( (hora * pi)/24 ) para agregarle periodicidad. 

In [ ]:
main_ahr_18_20 = rh_18_20.copy()
main_ahr_21_23 = rh_21_23.copy()

hour_mode_18_20 = auc_18_20[["device_id"]]; hour_mode_18_20["hour"] = pd.to_datetime( auc_18_20["date"] ).dt.hour
hour_mode_21_23 = auc_21_23[["device_id"]]; hour_mode_21_23["hour"] = pd.to_datetime( auc_21_23["date"] ).dt.hour

hour_mode_18_20 = hour_mode_18_20.groupby( ["device_id", "hour"] ).agg( { "hour":"count" } ).rename( columns = {"hour":"count"} ).reset_index()
hour_mode_21_23 = hour_mode_21_23.groupby( ["device_id", "hour"] ).agg( { "hour":"count" } ).rename( columns = {"hour":"count"} ).reset_index()

hour_mode_18_20 = hour_mode_18_20.sort_values( ["device_id","count"], ascending = False ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id":"ref_hash"} ).drop("count",axis = 1)
hour_mode_21_23 = hour_mode_21_23.sort_values( ["device_id","count"], ascending = False ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id":"ref_hash"} ).drop("count",axis = 1)

main_ahr_18_20 = main_ahr_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" ).rename( columns = {"hour":"main_hour"} )
main_ahr_21_23 = main_ahr_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" ).rename( columns = {"hour":"main_hour"} )

main_ahr_18_20["main_hour"] =  main_ahr_18_20["main_hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )
main_ahr_21_23["main_hour"] =  main_ahr_21_23["main_hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )

main_ahr_18_20.to_csv( loc_ftr + "\\main_ahr_18_20.csv", index = False )
main_ahr_21_23.to_csv( loc_ftr + "\\main_ahr_21_23.csv", index = False )

## <span style="color:green"> **Horario de la primer subasta en ese ventana** </span> 

In [ ]:
hr_f_auc_18_20 = rh_18_20.copy()
hr_f_auc_21_23 = rh_21_23.copy()

first_auc_hour_18_20 = auc_18_20[ ["device_id","date"] ].sort_values( "date" ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )
first_auc_hour_21_23 = auc_21_23[ ["device_id","date"] ].sort_values( "date" ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )

first_auc_hour_18_20["time_to_frt_auc"] = ( pd.to_datetime( first_auc_hour_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
first_auc_hour_21_23["time_to_frt_auc"] = ( pd.to_datetime( first_auc_hour_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

first_auc_hour_18_20["hour_frt_auc"] = pd.to_datetime( first_auc_hour_18_20["date"] ).dt.hour
first_auc_hour_21_23["hour_frt_auc"] = pd.to_datetime( first_auc_hour_21_23["date"] ).dt.hour

first_auc_hour_18_20.drop( ["date", "time_to_frt_auc"], axis = 1, inplace = True )
first_auc_hour_21_23.drop( ["date", "time_to_frt_auc"], axis = 1, inplace = True )

hr_f_auc_18_20 = hr_f_auc_18_20.merge( first_auc_hour_18_20, how = "left", on = "ref_hash" )
hr_f_auc_21_23 = hr_f_auc_21_23.merge( first_auc_hour_21_23, how = "left", on = "ref_hash" )

hr_f_auc_18_20.to_csv( loc_ftr + "\\hr_f_auc_18_20.csv", index = False )
hr_f_auc_21_23.to_csv( loc_ftr + "\\hr_f_auc_21_23.csv", index = False )

## <span style="color:red"> **Cantidad de Subastas por dispositivo en la ventana previa a la conversion** </span>

In [118]:
cant_auc_18_20 = rh_18_20.copy()
cant_auc_21_23 = rh_21_23.copy()

cant_auc_18_20 = cant_auc_18_20.merge( auc_18_20["device_id"].value_counts().to_frame().reset_index().rename( columns = {"device_id": "cant_auc", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_auc_21_23 = cant_auc_21_23.merge( auc_21_23["device_id"].value_counts().to_frame().reset_index().rename( columns = {"device_id": "cant_auc", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

cant_auc_18_20.fillna( 0, inplace = True )
cant_auc_21_23.fillna( 0, inplace = True )

cant_auc_18_20.to_csv( loc_ftr + "\\cant_auc_18_20.csv", index = False )
cant_auc_21_23.to_csv( loc_ftr + "\\cant_auc_21_23.csv", index = False )

## <span style="color:red"> **Tiempo hasta la primer auction en ese ventana** </span> 

Le asignamos cuanto tiempo, en la ventana del 18-20, tardo en realizar su primer auction

In [30]:
frst_auc_18_20 = rh_18_20.copy()
frst_auc_21_23 = rh_21_23.copy()

first_auctions_rh_ins_18_20 = auc_18_20[ ["device_id","date"] ].sort_values( "date" ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )
first_auctions_rh_ins_21_23 = auc_21_23[ ["device_id","date"] ].sort_values( "date" ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )

first_auctions_rh_ins_18_20["time_to_frt_auc"] = ( pd.to_datetime( first_auctions_rh_ins_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
first_auctions_rh_ins_21_23["time_to_frt_auc"] = ( pd.to_datetime( first_auctions_rh_ins_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

first_auctions_rh_ins_18_20.drop( "date", axis = 1, inplace = True )
first_auctions_rh_ins_21_23.drop( "date", axis = 1, inplace = True )

frst_auc_18_20 = frst_auc_18_20.merge( first_auctions_rh_ins_18_20, how = "left", on = "ref_hash" )
frst_auc_21_23 = frst_auc_21_23.merge( first_auctions_rh_ins_21_23, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
frst_auc_18_20.fillna( 3 * 24 * 3600, inplace = True )
frst_auc_21_23.fillna( 3 * 24 * 3600, inplace = True )

frst_auc_18_20.to_csv( loc_ftr + "\\frst_auc_18_20.csv", index = False )
frst_auc_21_23.to_csv( loc_ftr + "\\frst_auc_21_23.csv", index = False )

# <span style="color:green"> **Source que recibio mas instalaciones por dispositivo** </span> 

#### <span style="color:orange"> **Mean Encoding** </span> (Usamos el promedio de la cantidad de veces que que es la fuente principal de algun dispositivo) - <span style="color:red"> **No Mejora este tipo de codificacion** </span>

In [45]:
srce_auc_18_20 = rh_18_20.copy()
srce_auc_21_23 = rh_21_23.copy()

main_source_18_20 = auc_18_20.groupby( ["device_id", "source_id"] ).agg( {"source_id":"count"} ).rename( columns = {"source_id":"cant_auc"} ).reset_index()
main_source_21_23 = auc_21_23.groupby( ["device_id", "source_id"] ).agg( {"source_id":"count"} ).rename( columns = {"source_id":"cant_auc"} ).reset_index()

main_source_18_20 = main_source_18_20.sort_values( by = ["device_id","cant_auc"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "last" ); del( main_source_18_20["cant_auc"] )
main_source_21_23 = main_source_21_23.sort_values( by = ["device_id","cant_auc"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "last" ); del( main_source_21_23["cant_auc"] )

srce_auc_18_20 = srce_auc_18_20.merge( main_source_18_20.rename(columns = {"device_id":"ref_hash"}), how = "left", on = "ref_hash" ); srce_auc_18_20.fillna( "no_font", inplace = True )
srce_auc_21_23 = srce_auc_21_23.merge( main_source_21_23.rename(columns = {"device_id":"ref_hash"}), how = "left", on = "ref_hash" ); srce_auc_21_23.fillna( "no_font", inplace = True )

srce_auc_18_20["to_count"] = 1; srce_auc_18_20["source_id"] = srce_auc_18_20[["source_id", "to_count"]].groupby("source_id").transform( "sum" ) / len(srce_auc_18_20); del(srce_auc_18_20["to_count"])
srce_auc_21_23["to_count"] = 1; srce_auc_21_23["source_id"] = srce_auc_21_23[["source_id", "to_count"]].groupby("source_id").transform( "sum" ) / len(srce_auc_21_23); del(srce_auc_21_23["to_count"])

srce_auc_18_20.to_csv( loc_ftr + "\\srce_auc_18_20.csv", index = False )
srce_auc_21_23.to_csv( loc_ftr + "\\srce_auc_21_23.csv", index = False )

#### <span style="color:orange"> **OneHotEncoding** </span> 

In [46]:
srce_auc_18_20 = rh_18_20.copy()
srce_auc_21_23 = rh_21_23.copy()

main_source_18_20 = auc_18_20.groupby( ["device_id", "source_id"] ).agg( {"source_id":"count"} ).rename( columns = {"source_id":"cant_auc"} ).reset_index()
main_source_21_23 = auc_21_23.groupby( ["device_id", "source_id"] ).agg( {"source_id":"count"} ).rename( columns = {"source_id":"cant_auc"} ).reset_index()

main_source_18_20 = main_source_18_20.sort_values( by = ["device_id","cant_auc"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "last" ); del( main_source_18_20["cant_auc"] )
main_source_21_23 = main_source_21_23.sort_values( by = ["device_id","cant_auc"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "last" ); del( main_source_21_23["cant_auc"] )

srce_auc_18_20 = srce_auc_18_20.merge( main_source_18_20.rename(columns = {"device_id":"ref_hash"}), how = "left", on = "ref_hash" ); srce_auc_18_20.fillna( "no_font", inplace = True )
srce_auc_21_23 = srce_auc_21_23.merge( main_source_21_23.rename(columns = {"device_id":"ref_hash"}), how = "left", on = "ref_hash" ); srce_auc_21_23.fillna( "no_font", inplace = True )

#Ya fue controlado que quedan las mismas fuentes para ambas ventanas, no quedan columnas distintas
srce_auc_ohe_18_20 = pd.get_dummies( srce_auc_18_20[ ["source_id"] ].astype( "category" ) )
srce_auc_ohe_21_23 = pd.get_dummies( srce_auc_21_23[ ["source_id"] ].astype( "category" ) )

srce_auc_18_20 = pd.concat( [srce_auc_18_20, srce_auc_ohe_18_20], axis = 1 ); del( srce_auc_18_20["source_id"] )
srce_auc_21_23 = pd.concat( [srce_auc_21_23, srce_auc_ohe_21_23], axis = 1 ); del( srce_auc_21_23["source_id"] )

srce_auc_18_20.to_csv( loc_ftr + "\\srce_auc_18_20.csv", index = False )
srce_auc_21_23.to_csv( loc_ftr + "\\srce_auc_21_23.csv", index = False )

## <span style="color:red"> **Tiempo hasta la ultima auction en esa ventana** </span> 

In [11]:
last_auc_18_20 = rh_18_20.copy()
last_auc_21_23 = rh_21_23.copy()

last_auctions_rh_ins_18_20 = auc_18_20[ ["device_id","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )
last_auctions_rh_ins_21_23 = auc_21_23[ ["device_id","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )

last_auctions_rh_ins_18_20["time_to_lst_auc"] = ( pd.to_datetime( last_auctions_rh_ins_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
last_auctions_rh_ins_21_23["time_to_lst_auc"] = ( pd.to_datetime( last_auctions_rh_ins_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

last_auctions_rh_ins_18_20.drop( "date", axis = 1, inplace = True )
last_auctions_rh_ins_21_23.drop( "date", axis = 1, inplace = True )

last_auc_18_20 = last_auc_18_20.merge( last_auctions_rh_ins_18_20, how = "left", on = "ref_hash" )
last_auc_21_23 = last_auc_21_23.merge( last_auctions_rh_ins_21_23, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
last_auc_18_20.fillna( 3 * 24 * 3600, inplace = True )
last_auc_21_23.fillna( 3 * 24 * 3600, inplace = True )

last_auc_18_20.to_csv( loc_ftr + "\\last_auc_18_20.csv", index = False )
last_auc_21_23.to_csv( loc_ftr + "\\last_auc_21_23.csv", index = False )

## <span style="color:red"> **Participo en mas de una subasta** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [70]:
cant_ins_18_20 = rh_18_20.copy()
cant_ins_21_23 = rh_21_23.copy()

cant_ins_18_20 = cant_ins_18_20.merge( ins_18_20["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "mt_1_ins", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_ins_21_23 = cant_ins_21_23.merge( ins_21_23["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "mt_1_ins", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

cant_ins_18_20.fillna( 0, inplace = True )
cant_ins_21_23.fillna( 0, inplace = True )

cant_ins_18_20['mt_1_ins'] = (cant_ins_18_20['mt_1_ins'] > 1).astype('int8')
cant_ins_21_23['mt_1_ins'] = (cant_ins_21_23['mt_1_ins'] > 1).astype('int8')

cant_ins_18_20['me_mt_1_ins'] = ( cant_ins_18_20.groupby('mt_1_ins').transform('count') ) / len(cant_ins_18_20['mt_1_ins'])
cant_ins_21_23['me_mt_1_ins'] = ( cant_ins_21_23.groupby('mt_1_ins').transform('count') ) / len(cant_ins_21_23['mt_1_ins'])

del cant_ins_18_20['mt_1_ins']
del cant_ins_21_23['mt_1_ins']

cant_ins_18_20.to_csv( loc_ftr + "\\m1me_auc_18_20.csv", index = False )
cant_ins_21_23.to_csv( loc_ftr + "\\m1me_auc_21_23.csv", index = False )

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [71]:
cant_ins_18_20 = rh_18_20.copy()
cant_ins_21_23 = rh_21_23.copy()

cant_ins_18_20 = cant_ins_18_20.merge( ins_18_20["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "mt_1_ins", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_ins_21_23 = cant_ins_21_23.merge( ins_21_23["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "mt_1_ins", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

cant_ins_18_20.fillna( 0, inplace = True )
cant_ins_21_23.fillna( 0, inplace = True )

cant_ins_18_20['mt_1_ins'] = (cant_ins_18_20['mt_1_ins'] > 1).astype('int8')
cant_ins_21_23['mt_1_ins'] = (cant_ins_21_23['mt_1_ins'] > 1).astype('int8')

cant_ins_18_20.to_csv( loc_ftr + "\\m1oh_auc_18_20.csv", index = False )
cant_ins_21_23.to_csv( loc_ftr + "\\m1oh_auc_21_23.csv", index = False )

## <span style="color:green"> **Recibio subasta entre 21 hs y 3 hs (Noche)** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [45]:
auc_18_20['auc_21_3'] = ( pd.to_datetime( auc_18_20['date'] ).dt.hour < 4) | ( pd.to_datetime( auc_18_20['date'] ).dt.hour > 20)
auc_21_23['auc_21_3'] = ( pd.to_datetime( auc_21_23['date'] ).dt.hour < 4) | ( pd.to_datetime( auc_21_23['date'] ).dt.hour > 20)

auc_night_18_20 = rh_18_20.copy()
auc_night_21_23 = rh_21_23.copy()

hour_mode_18_20 = auc_18_20.groupby('device_id').agg({'auc_21_3':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_21_23 = auc_21_23.groupby('device_id').agg({'auc_21_3':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_night_18_20 = auc_night_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
auc_night_21_23 = auc_night_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

auc_night_18_20["auc_21_3"] =  (auc_night_18_20["auc_21_3"] > 1).astype('int8')
auc_night_21_23["auc_21_3"] =  (auc_night_21_23["auc_21_3"] > 1).astype('int8')

auc_night_18_20['me_auc_21_3'] = ( auc_night_18_20.groupby('auc_21_3').transform('count') ) / len(auc_night_18_20['auc_21_3'])
auc_night_21_23['me_auc_21_3'] = ( auc_night_21_23.groupby('auc_21_3').transform('count') ) / len(auc_night_21_23['auc_21_3'])

del auc_night_18_20["auc_21_3"]
del auc_night_21_23["auc_21_3"]

auc_night_18_20.to_csv( loc_ftr + "\\auc_ngme_18_20.csv", index = False )
auc_night_21_23.to_csv( loc_ftr + "\\auc_ngme_21_23.csv", index = False )

del auc_18_20['auc_21_3']
del auc_21_23['auc_21_3']

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [46]:
auc_18_20['auc_21_3'] = ( pd.to_datetime( auc_18_20['date'] ).dt.hour < 4) | ( pd.to_datetime( auc_18_20['date'] ).dt.hour > 20)
auc_21_23['auc_21_3'] = ( pd.to_datetime( auc_21_23['date'] ).dt.hour < 4) | ( pd.to_datetime( auc_21_23['date'] ).dt.hour > 20)

auc_night_18_20 = rh_18_20.copy()
auc_night_21_23 = rh_21_23.copy()

hour_mode_18_20 = auc_18_20.groupby('device_id').agg({'auc_21_3':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_21_23 = auc_21_23.groupby('device_id').agg({'auc_21_3':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_night_18_20 = auc_night_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
auc_night_21_23 = auc_night_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

auc_night_18_20["auc_21_3"] =  (auc_night_18_20["auc_21_3"] > 1).astype('int8')
auc_night_21_23["auc_21_3"] =  (auc_night_21_23["auc_21_3"] > 1).astype('int8')

auc_night_18_20.to_csv( loc_ftr + "\\auc_ngoh_18_20.csv", index = False )
auc_night_21_23.to_csv( loc_ftr + "\\auc_ngoh_21_23.csv", index = False )

del auc_18_20['auc_21_3']
del auc_21_23['auc_21_3']

## <span style="color:green"> **Recibio subasta entre 4 hs y 10 hs (Mañana)** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [47]:
auc_18_20['auc_4_10'] = ( pd.to_datetime( auc_18_20['date'] ).dt.hour < 11) & ( pd.to_datetime( auc_18_20['date'] ).dt.hour > 3)
auc_21_23['auc_4_10'] = ( pd.to_datetime( auc_21_23['date'] ).dt.hour < 11) & ( pd.to_datetime( auc_21_23['date'] ).dt.hour > 3)

auc_morn_18_20 = rh_18_20.copy()
auc_morn_21_23 = rh_21_23.copy()

hour_mode_18_20 = auc_18_20.groupby('device_id').agg({'auc_4_10':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_21_23 = auc_21_23.groupby('device_id').agg({'auc_4_10':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_morn_18_20 = auc_morn_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
auc_morn_21_23 = auc_morn_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

auc_morn_18_20["auc_4_10"] =  (auc_morn_18_20["auc_4_10"] > 1).astype('int8')
auc_morn_21_23["auc_4_10"] =  (auc_morn_21_23["auc_4_10"] > 1).astype('int8')

auc_morn_18_20['me_auc_4_10'] = ( auc_morn_18_20.groupby('auc_4_10').transform('count') ) / len(auc_morn_18_20['auc_4_10'])
auc_morn_21_23['me_auc_4_10'] = ( auc_morn_21_23.groupby('auc_4_10').transform('count') ) / len(auc_morn_21_23['auc_4_10'])

del auc_morn_18_20["auc_4_10"]
del auc_morn_21_23["auc_4_10"]

auc_morn_18_20.to_csv( loc_ftr + "\\auc_mrme_18_20.csv", index = False )
auc_morn_21_23.to_csv( loc_ftr + "\\auc_mrme_21_23.csv", index = False )

del auc_18_20['auc_4_10']
del auc_21_23['auc_4_10']

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [48]:
auc_18_20['auc_4_10'] = ( pd.to_datetime( auc_18_20['date'] ).dt.hour < 11) & ( pd.to_datetime( auc_18_20['date'] ).dt.hour > 3)
auc_21_23['auc_4_10'] = ( pd.to_datetime( auc_21_23['date'] ).dt.hour < 11) & ( pd.to_datetime( auc_21_23['date'] ).dt.hour > 3)

auc_morn_18_20 = rh_18_20.copy()
auc_morn_21_23 = rh_21_23.copy()

hour_mode_18_20 = auc_18_20.groupby('device_id').agg({'auc_4_10':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_21_23 = auc_21_23.groupby('device_id').agg({'auc_4_10':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_morn_18_20 = auc_morn_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
auc_morn_21_23 = auc_morn_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

auc_morn_18_20["auc_4_10"] =  (auc_morn_18_20["auc_4_10"] > 1).astype('int8')
auc_morn_21_23["auc_4_10"] =  (auc_morn_21_23["auc_4_10"] > 1).astype('int8')

auc_morn_18_20.to_csv( loc_ftr + "\\auc_mroh_18_20.csv", index = False )
auc_morn_21_23.to_csv( loc_ftr + "\\auc_mroh_21_23.csv", index = False )

del auc_18_20['auc_4_10']
del auc_21_23['auc_4_10']

## <span style="color:green"> **Recibio subasta entre 11 hs y 15 hs (Mediodia)** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [49]:
auc_18_20['auc_11_15'] = ( pd.to_datetime( auc_18_20['date'] ).dt.hour < 16) & ( pd.to_datetime( auc_18_20['date'] ).dt.hour > 10)
auc_21_23['auc_11_15'] = ( pd.to_datetime( auc_21_23['date'] ).dt.hour < 16) & ( pd.to_datetime( auc_21_23['date'] ).dt.hour > 10)

auc_midday_18_20 = rh_18_20.copy()
auc_midday_21_23 = rh_21_23.copy()

hour_mode_18_20 = auc_18_20.groupby('device_id').agg({'auc_11_15':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_21_23 = auc_21_23.groupby('device_id').agg({'auc_11_15':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_midday_18_20 = auc_midday_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
auc_midday_21_23 = auc_midday_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

auc_midday_18_20["auc_11_15"] =  (auc_midday_18_20["auc_11_15"] > 1).astype('int8')
auc_midday_21_23["auc_11_15"] =  (auc_midday_21_23["auc_11_15"] > 1).astype('int8')

auc_midday_18_20['me_auc_11_15'] = ( auc_midday_18_20.groupby('auc_11_15').transform('count') ) / len(auc_midday_18_20['auc_11_15'])
auc_midday_21_23['me_auc_11_15'] = ( auc_midday_21_23.groupby('auc_11_15').transform('count') ) / len(auc_midday_21_23['auc_11_15'])

del auc_midday_18_20["auc_11_15"]
del auc_midday_21_23["auc_11_15"]

auc_midday_18_20.to_csv( loc_ftr + "\\auc_mdme_18_20.csv", index = False )
auc_midday_21_23.to_csv( loc_ftr + "\\auc_mdme_21_23.csv", index = False )

del auc_18_20['auc_11_15']
del auc_21_23['auc_11_15']

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [50]:
auc_18_20['auc_11_15'] = ( pd.to_datetime( auc_18_20['date'] ).dt.hour < 16) & ( pd.to_datetime( auc_18_20['date'] ).dt.hour > 10)
auc_21_23['auc_11_15'] = ( pd.to_datetime( auc_21_23['date'] ).dt.hour < 16) & ( pd.to_datetime( auc_21_23['date'] ).dt.hour > 10)

auc_midday_18_20 = rh_18_20.copy()
auc_midday_21_23 = rh_21_23.copy()

hour_mode_18_20 = auc_18_20.groupby('device_id').agg({'auc_11_15':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_21_23 = auc_21_23.groupby('device_id').agg({'auc_11_15':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_midday_18_20 = auc_midday_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
auc_midday_21_23 = auc_midday_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

auc_midday_18_20["auc_11_15"] =  (auc_midday_18_20["auc_11_15"] > 1).astype('int8')
auc_midday_21_23["auc_11_15"] =  (auc_midday_21_23["auc_11_15"] > 1).astype('int8')

auc_midday_18_20.to_csv( loc_ftr + "\\auc_mdoh_18_20.csv", index = False )
auc_midday_21_23.to_csv( loc_ftr + "\\auc_mdoh_21_23.csv", index = False )

del auc_18_20['auc_11_15']
del auc_21_23['auc_11_15']

## <span style="color:green"> **Recibio subasta entre 16 hs y 20 hs (Tarde)** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [51]:
auc_18_20['auc_16_20'] = ( pd.to_datetime( auc_18_20['date'] ).dt.hour < 21) & ( pd.to_datetime( auc_18_20['date'] ).dt.hour > 15)
auc_21_23['auc_16_20'] = ( pd.to_datetime( auc_21_23['date'] ).dt.hour < 21) & ( pd.to_datetime( auc_21_23['date'] ).dt.hour > 15)

auc_after_18_20 = rh_18_20.copy()
auc_after_21_23 = rh_21_23.copy()

hour_mode_18_20 = auc_18_20.groupby('device_id').agg({'auc_16_20':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_21_23 = auc_21_23.groupby('device_id').agg({'auc_16_20':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_after_18_20 = auc_after_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
auc_after_21_23 = auc_after_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

auc_after_18_20["auc_16_20"] =  (auc_after_18_20["auc_16_20"] > 1).astype('int8')
auc_after_21_23["auc_16_20"] =  (auc_after_21_23["auc_16_20"] > 1).astype('int8')

auc_after_18_20['me_auc_16_20'] = ( auc_after_18_20.groupby('auc_16_20').transform('count') ) / len(auc_after_18_20['auc_16_20'])
auc_after_21_23['me_auc_16_20'] = ( auc_after_21_23.groupby('auc_16_20').transform('count') ) / len(auc_after_21_23['auc_16_20'])

del auc_after_18_20["auc_16_20"]
del auc_after_21_23["auc_16_20"]

auc_after_18_20.to_csv( loc_ftr + "\\auc_afme_18_20.csv", index = False )
auc_after_21_23.to_csv( loc_ftr + "\\auc_afme_21_23.csv", index = False )

del auc_18_20['auc_16_20']
del auc_21_23['auc_16_20']

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [52]:
auc_18_20['auc_16_20'] = ( pd.to_datetime( auc_18_20['date'] ).dt.hour < 21) & ( pd.to_datetime( auc_18_20['date'] ).dt.hour > 15)
auc_21_23['auc_16_20'] = ( pd.to_datetime( auc_21_23['date'] ).dt.hour < 21) & ( pd.to_datetime( auc_21_23['date'] ).dt.hour > 15)

auc_after_18_20 = rh_18_20.copy()
auc_after_21_23 = rh_21_23.copy()

hour_mode_18_20 = auc_18_20.groupby('device_id').agg({'auc_16_20':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_21_23 = auc_21_23.groupby('device_id').agg({'auc_16_20':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_after_18_20 = auc_after_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
auc_after_21_23 = auc_after_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

auc_after_18_20["auc_16_20"] =  (auc_after_18_20["auc_16_20"] > 1).astype('int8')
auc_after_21_23["auc_16_20"] =  (auc_after_21_23["auc_16_20"] > 1).astype('int8')

auc_after_18_20.to_csv( loc_ftr + "\\auc_afoh_18_20.csv", index = False )
auc_after_21_23.to_csv( loc_ftr + "\\auc_afoh_21_23.csv", index = False )

del auc_18_20['auc_16_20']
del auc_21_23['auc_16_20']